In [ ]:
# !pip install openai
# !pip install python-dotenv

In [65]:
import os
import json
import time
import openai
import sqlite3
import pandas as pd

from datetime import datetime
from dotenv import load_dotenv
from webdriver_manager.chrome import ChromeDriverManager 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [18]:
# Load the variables from .env into the environment
load_dotenv()

True

## Connecting to the News Website

In [66]:
URL = "https://www.maritimelogisticsprofessional.com"

In [67]:
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) 
browser.implicitly_wait(5)
browser.get(URL)

In [68]:
try:
    # Wait for the button to be clickable
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='Ok']"))
    )
    # Click the button once it is clickable
    browser.find_element(By.XPATH, "//button[text()='Ok']").click()
except (NoSuchElementException, TimeoutException):
    print("The cookie acceptance button was not found on the page.")

In [53]:
# List of latest news
latest_news = browser.find_elements(By.CLASS_NAME, "snippet-flex")
latest_news

[<selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f66296", element="f.1F6728003F0A99EF7C8BEE4B1F22756D.d.3AB28E677831FE7E401C233B0EEBA3DA.e.31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f66296", element="f.1F6728003F0A99EF7C8BEE4B1F22756D.d.3AB28E677831FE7E401C233B0EEBA3DA.e.6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f66296", element="f.1F6728003F0A99EF7C8BEE4B1F22756D.d.3AB28E677831FE7E401C233B0EEBA3DA.e.9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f66296", element="f.1F6728003F0A99EF7C8BEE4B1F22756D.d.3AB28E677831FE7E401C233B0EEBA3DA.e.32")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f66296", element="f.1F6728003F0A99EF7C8BEE4B1F22756D.d.3AB28E677831FE7E401C233B0EEBA3DA.e.33")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc13b25fbeeaaa07438b45a3b3f

In [28]:
# Article
article_link = latest_news[0].get_attribute('href')
article_link

'https://www.maritimeprofessional.com/news/houthi-rockets-wreak-havoc-middle-390817'

In [23]:
# Get article
time.sleep(5)
browser.get(article_link)

In [24]:
# Find the title element and get the text
title_element = browser.find_element(By.CSS_SELECTOR, "h1[itemprop='name']")
article_title = title_element.text

# Find the article body element and get all the paragraph texts
article_body_element = browser.find_element(By.CSS_SELECTOR, "div[property='articleBody']")
article_paragraphs = article_body_element.find_elements(By.TAG_NAME, "p")

# Combine the text of all paragraphs to form the body text
article_text = " ".join(paragraph.text for paragraph in article_paragraphs)

# Now you have the article's title and text
print("Title:", article_title)
print("Article Text:", article_text)

Title: Port of Long Beach Opens New Fireboat Stations
Article Text: Officials dedicated two new Port of Long Beach fireboat stations that are enhancing the Long Beach Fire Department’s waterside and landside emergency response capabilities, better safeguarding visiting ships, cargo and waterfront workers. The facilities – Fireboat Station 15 and Fireboat Station 20 – are the products of a $109 million Port of Long Beach program to preserve business continuity, security and economic interests. With an anticipated lifespan of 50 years, both stations were approved for construction in 2017 by the Long Beach Board of Harbor Commissioners and funded by Port revenues. “These fireboat stations will provide an important and invaluable safety service to our Port,” said Long Beach Mayor Rex Richardson. “Our firefighters are now equipped with the most sophisticated facilities and emergency response capabilities to protect our Port, its valuable assets and our waterfront workforce.” “This is a grea

In [ ]:
# Collect all URLs before navigating
urls = [element.get_attribute('href') for element in latest_news]

for url in urls:
    print(f"Getting {url}")
    time.sleep(5)  # Consider using WebDriverWait here instead of time.sleep for better efficiency
    browser.get(url)
    # Now you can perform your scraping logic on each article page

In [69]:
browser.quit()

---

## Categorizing the Article

In [ ]:
# Load keywords from JSON file
with open('../json/keywords.json', 'r') as file:
    keywords = json.load(file)

def classify_article(article_text, keywords):
    max_count = 0
    max_category = "Unclassified or Neutral"

    # Convert article text to lower case for comparison
    article_text_lower = article_text.lower()
    
    # Check for the presence of each keyword in the article
    for category, category_keywords in keywords.items():
        count = sum(keyword in article_text_lower for keyword in category_keywords)
        if count > max_count:
            max_count = count
            max_category = category

    return max_category

# Usage example:
classification = classify_article(article_text, keywords)
print(f"The article is related to: {classification}")


---

## Connect to the OpenAI API

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

def summarize_text(text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt= f"""
Please summarize the key points of the article for a business audience in a concise paragraph, limiting the summary to no more than 350 characters. Then, in a separate paragraph of 500 characters, elaborate on the potential impact of the situation described in the article on maritime logistics, port operations, and supply chain management, specifically focusing on its implications for Latin America. Label this second paragraph 'Impacto en LATAM:' and ensure there is a clear separation between the two sections. Present your response in Spanish and format it as markdown text for clarity:\n\n{text}
                """,
        temperature=0.7,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].text.strip()

#summary = summarize_text(article_text)

In [ ]:
summary = "Get Premium for enabling AI-powered summary!"

---

## Save Original Article, Title and Summary in DDBB

In [ ]:
# Establishes a connection to the SQLite database
def connect_to_db(db_path):
    return sqlite3.connect(db_path)

# Create a new table for storing only daily articles
def create_daily_table(cursor, table_name):
    """
    """
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        text TEXT,
                        summary TEXT,
                        classification TEXT
                    );""")

# Create a new table for storing daily links
def create_daily_links_table(cursor, table_name, articles_table_name):
    """
    """
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        article_id INTEGER,
                        title TEXT,
                        link TEXT,
                        FOREIGN KEY (article_id) REFERENCES {articles_table_name}(id)
                    );""")

# Inserts an article into the daily table
def insert_article_data(cursor, table_name, article_title, article_text, summary, classification):
    """
    """
    # Check if an article with the same title already exists
    cursor.execute(f"SELECT id FROM {table_name} WHERE title = ?", (article_title,))
    existing_article = cursor.fetchone()

    if existing_article == None:
        cursor.execute(f"""INSERT INTO {table_name} (title, text, summary, classification)
                        VALUES (?, ?, ?, ?);""",
                    (article_title, article_text, summary, classification))
        return cursor.lastrowid  # Return the ID of the new article
    else:
        return existing_article[0]  # Return the ID of the existing article

# Inserts a link into the daily links table
def insert_link_data(cursor, table_name, article_id, article_title, link):
    """
    """
    # Check if the link already exists for the given article
    cursor.execute(f"SELECT id FROM {table_name} WHERE article_id = ? AND link = ?", (article_id, link))
    existing_link = cursor.fetchone()

    if existing_link is None:
        # If the link does not exist for this article, insert it
        cursor.execute(f"""INSERT INTO {table_name} (article_id, title, link)
                           VALUES (?, ?, ?);""",
                       (article_id, article_title, link))

### DDBB Initialization

In [ ]:
# Define the path to the SQLite database
db_path = '../data/news/maritime_news.db'

# Define table naming
current_date = datetime.now().strftime("%m%d%Y")

# Establish a connection and create a cursor
conn = connect_to_db(db_path)
cursor = conn.cursor()

# Create tables for today's date with news
news_table_name = f"news_{current_date}"
create_daily_table(cursor, news_table_name)

# Create table for storing links
links_table_name = f"news_links_{current_date}"
create_daily_links_table(cursor, links_table_name, news_table_name)

# Commit the changes and close the connection
conn.commit()
conn.close()

### DDBB News/Links Input or Update

In [ ]:
# Establish a connection and create a cursor
conn = connect_to_db(db_path)
cursor = conn.cursor()

# Insert the article data into the table and get the article ID
create_daily_table(cursor, news_table_name)
article_id = insert_article_data(cursor, news_table_name, article_title, article_text, summary, classification)

# Insert the link data into the links table
insert_link_data(cursor, links_table_name, article_id, article_title, article_link)

# Commit the changes and close the connection
conn.commit()
conn.close()

---

## Full Cycle

In [135]:
# Create a browser session
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) 
browser.implicitly_wait(2)
browser.get(URL)

# Cookies button
try:
    # Wait for the button to be clickable
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='Ok']"))
    )
    # Click the button once it is clickable
    browser.find_element(By.XPATH, "//button[text()='Ok']").click()
except (NoSuchElementException, TimeoutException):
    print("The cookie acceptance button was not found on the page.")

In [ ]:
"""
GLOBAL LATEST NEWS
"""

# List of latest global news
latest_news = browser.find_elements(By.CLASS_NAME, "snippet-flex")
# Collect all URLs before navigating
article_urls = [element.get_attribute('href') for element in latest_news]
# Premium version
premium = False

# Establish a connection and create a cursor
conn = connect_to_db(db_path)
cursor = conn.cursor()

for article in article_urls:
    """
    """
    # Get article
    time.sleep(5)
    browser.get(article)

    # Find the title element and get the text
    title_element = browser.find_element(By.CSS_SELECTOR, "h1[itemprop='name']")
    article_title = title_element.text

    # Find the article body element and get all the paragraph texts
    article_body_element = browser.find_element(By.CSS_SELECTOR, "div[property='articleBody']")
    article_paragraphs = article_body_element.find_elements(By.TAG_NAME, "p")

    # Combine the text of all paragraphs to form the body text
    article_text = " ".join(paragraph.text for paragraph in article_paragraphs)

    # AI-powered Summary
    if premium:
        summarize_text(article_text)
    else:
        # Summary not premium
        summary = "Get Premium for enabling AI-powered summary!"
    
    # Article classification
    classification = classify_article(article_text, keywords)

    # Insert the article data into the table and get the article ID
    article_id = insert_article_data(cursor, news_table_name, article_title, article_text, summary, classification)
    # Insert the link data into the links table
    insert_link_data(cursor, links_table_name, article_id, article_title, article_link)

    # Commit the changes
    conn.commit()

# Close browser and database connection
browser.quit()
conn.close()

In [136]:
"""
SOUTH AMERICA NEWS
"""
filter_news = '/south-america'

# Access the '.cat-link' (category links) elements
cat_links = browser.find_elements(By.CSS_SELECTOR, ".cat-link")

# Iterate through each 'cat-link' div
for cat_link_div in cat_links:
    # Find the 'a' element within the current 'cat-link' div
    a_element = cat_link_div.find_element(By.TAG_NAME, 'a')  # Note: it's 'find_element' not 'find_elements'
    # Get the 'href' attribute from the 'a' element
    href = a_element.get_attribute('href')
    if href.endswith(f"/{filter_news}"):
        # Print the href
        print(href)
